<h3>Testing the temporal entity recogniser</h3>

In [9]:
import os
os.listdir()

['TestTemporalEntityRecogniser.ipynb',
 'periodo-cache.json',
 'periodo_cache.json',
 'tokens.txt']

In [3]:
from pprint import pprint
pprint(sys.path)

['/workspace/rematch2/notebook',
 '/home/gitpod/.pyenv/versions/3.8.13/lib/python38.zip',
 '/home/gitpod/.pyenv/versions/3.8.13/lib/python3.8',
 '/home/gitpod/.pyenv/versions/3.8.13/lib/python3.8/lib-dynload',
 '',
 '/home/gitpod/.pyenv/versions/3.8.13/lib/python3.8/site-packages',
 '/workspace/rematch2/src']


In [2]:
%load_ext autoreload
%autoreload 2

import json
import ipywidgets as widgets

# tell Python where the local modules are located
import sys
module_path = os.path.abspath(os.path.join('..', 'src'))
if module_path not in sys.path:
    sys.path.append(module_path)

# import local modules
from Recognizer import Recognizer
from PeriodoData import PeriodoData

# modules for UI
from IPython.display import display, HTML


# get list of all Perio.do authorities
pd = PeriodoData(False) # create new instance, don't refresh cache
authorities = pd.get_authority_list() # returns [{id: "x", label: "label x"}, {id: "y", label: "label y"}]
authorities_for_dropdown = list(map(lambda x: [x["label"], x["id"]], authorities)) # dropdown requires array of tuples: [[label, id], [label, id]]

# define dropdown control for list of Perio.do authorities
dropdown_authorities = widgets.Dropdown(
    options = authorities_for_dropdown, 
    value=None, 
    description='Authority:',
    disabled=False
)

# define dropdown control for list of Perio.do periods
dropdown_periods = widgets.Dropdown(
    options = [], 
    value=None, 
    description='Periods:',
    disabled=False
)

# what to do when selected perio.do authority is changed
def on_dropdown_change(*args):
    periods = pd.get_period_list(dropdown_authorities.value)
    periods_for_dropdown = list(map(lambda item: [item["label"], item["id"]], periods))
    dropdown_periods.options = periods_for_dropdown

dropdown_authorities.observe(on_dropdown_change, 'value')


# define input textarea
example_input_text = "the artefact was medieval or circa 1250-1275 or maybe post medieval? Underlying the modern made ground on the site was a layer covering the entire shaft area. This has been dated to c.1480-1800/1900 and interpreted as a post-medieval cultivation soil. Historic mapping illustrates that the site remained undeveloped through the post medieval period until the mid 19th century, when urban development around the site accelerated and construction of railways in this part of London began. On Gascoigne's 1703 map the site was open ground, the later maps of Rocque in 1746 and Horwood in 1799 show the area was in use as fields and Stanford's map of 1862 depicts the area surrounding Eleanor Street comprising of market gardens. These are all consistent with the archaeological evidence. Underlying the layer were natural terrace gravels. The archaeological fieldwork has demonstrated that remains relating to the Prehistoric, Roman or medieval period have not survived to the modern era, if they were once present on site."

textarea_input = widgets.Textarea(
    value=example_input_text,
    placeholder='Type something',
    description='String:',
    disabled=False,
    layout=widgets.Layout(width='100%', height='150px')
    
)

# define output format dropdown
dropdown_output_format = widgets.Dropdown(
    options = ["html", "json", "CSV", "TSV"], 
    value="html", 
    description='Format:',
    disabled=False
)

# define 'Go' button and output area
button_go = widgets.Button(description="Go")
output = widgets.Output(
    layout=widgets.Layout(overflow='scroll', border='1px solid black', height='150px')
)

def on_button_go_clicked(b):
    
    language = "en"
    input_text = txt_input.value
    output_format = dropdown_output_format.value
    ner = TemporalEntityRecognizer(language)
    entities = ner.get_entities(input_text)
    formatted = TemporalEntityRecognizer.format_entities(input_text, entities, output_format)
    print(formatted)
    with output:
        pprint(formatted)
        #output.clear_output(wait=True)
        display(HTML(formatted))
        #display(HTML('<a href="http://example.com">link</a>'))
    #display(formatted)
        

button_go.on_click(on_button_go_clicked)
    
# display all controls on the screen

display(textarea_input)
display(dropdown_authorities)
display(dropdown_periods)
display(dropdown_output_format)
display(button_go)
display(output) 
#display(HTML('<a href="http://example.com">link</a>'))


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Textarea(value="the artefact was medieval or circa 1250-1275 or maybe post medieval? Underlying the modern mad…

Dropdown(description='Authority:', options=([' Alfarería arqueológica del centro sur de Chile y la Patagonia N…

Dropdown(description='Periods:', options=(), value=None)

Dropdown(description='Format:', options=('html', 'json', 'CSV', 'TSV'), value='html')

Button(description='Go', style=ButtonStyle())

Output(layout=Layout(border='1px solid black', height='150px', overflow='scroll'))